# NBA MVP Prediction Using Machine Learning
This notebook follows the same structure as `mvp_prediction.py`, loading data, preprocessing, training three models (Logistic Regression, Random Forest, XGBoost), evaluating them, plotting ROC curves, and running SHAP interpretability.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
    roc_curve, 
    make_scorer, f1_score, roc_auc_score
)

# Load data
mvp = pd.read_csv("data/mvp_history.csv", skiprows=1, usecols=["Season", "Player"])
mvp["season"] = mvp["Season"].str.split("-", expand=True)[0].astype(int) + 1
mvp = mvp.rename(columns={"Player": "player"})[["season", "player"]].assign(MVP=1)

bb = pd.read_csv("data/NBA_Dataset.csv")
rap_h = pd.read_csv("data/historical_RAPTOR_by_player.csv")
rap_m = pd.read_csv("data/modern_RAPTOR_by_player.csv")
rap_cols = ["player_name", "season", "raptor_offense", "raptor_defense", "raptor_total"]
rap = pd.concat([rap_h[rap_cols], rap_m[rap_cols]], ignore_index=True)
rap = rap.drop_duplicates(subset=["player_name", "season"]).rename(columns={"player_name": "player"})

df = bb.merge(rap, on=["season", "player"], how="left").merge(mvp, on=["season", "player"], how="left")
df["MVP"] = df["MVP"].fillna(0).astype(int)
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())


In [ ]:
# Feature engineering
features = [
    'ws', 'bpm', 'vorp', 'win_loss_pct',
    'pts_per_g', 'trb_per_g', 'ast_per_g',
    'raptor_offense', 'raptor_defense', 'raptor_total'
]
X = df[features]
y = df["MVP"]


In [ ]:
# Train/test split
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# Train models
models = {
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(max_iter=2000, random_state=42))
    ]).fit(X_tr, y_tr),
    "RandomForest": RandomForestClassifier(
        n_estimators=300, max_depth=8, random_state=42, n_jobs=1
    ).fit(X_tr, y_tr),
    "XGBoost": xgb.XGBClassifier(
        n_estimators=300, learning_rate=0.05,
        eval_metric="logloss",
        random_state=42, n_jobs=1
    ).fit(X_tr, y_tr)
}


In [ ]:
# Evaluation & Reporting
for name, model in models.items():
    y_pred = model.predict(X_te)
    y_prob = model.predict_proba(X_te)[:, 1]
    print(f"=== {name} ===")
    print(classification_report(y_te, y_pred, digits=3))
    print(f"ROC AUC: {roc_auc_score(y_te, y_prob):.3f}")
    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
    print(f"Confusion Matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}\n")


In [ ]:
# Plot ROC curves
plt.figure()
for name, model in models.items():
    probas = model.predict_proba(X_te)[:, 1]
    fpr, tpr, _ = roc_curve(y_te, probas)
    auc = roc_auc_score(y_te, probas)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")
plt.plot([0, 1], [0, 1], "--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for MVP Prediction Models")
plt.legend()
plt.show()


In [ ]:
for name, model in models.items():
    if name == "XGBoost":
        booster = model.get_booster()
        score = booster.get_score(importance_type="gain")
        importances = pd.Series(score).reindex(features).fillna(0)
    elif hasattr(model, "feature_importances_"):
        importances = pd.Series(model.feature_importances_, index=features)
    else:
        print(f"Skipping {name} (no native feature importance)")
        continue

    importances = importances.sort_values(ascending=False)
    plt.figure(figsize=(8, 6))
    importances.plot(kind="barh")
    plt.title(f"{name} Feature Importance")
    plt.xlabel("Importance Score")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

In [ ]:
# SHAP interpretability for Logistic Regression
scaler = models["LogisticRegression"].named_steps["scaler"]
estimator = models["LogisticRegression"].named_steps["lr"]
X_scaled = scaler.transform(X_tr)

explainer = shap.Explainer(estimator.predict, masker=shap.maskers.Independent(X_scaled))
shap_vals = explainer(X_scaled)
shap.summary_plot(
    shap_vals, 
    X_scaled, 
    feature_names=features, 
    show=False
)
plt.title("SHAP Summary — Logistic Regression")
plt.show()

In [ ]:
# --- Random Forest SHAP ---
explainer_rf      = shap.TreeExplainer(models["RandomForest"])
shap_vals_rf_all  = explainer_rf.shap_values(X_scaled)  

# pick out the “positive” class (class index 1) across the third axis
shap_vals_rf = shap_vals_rf_all[:, :, 1]  

# now shap_vals_rf has shape (n_samples, n_features), so it will line up
shap.summary_plot(
    shap_vals_rf,
    X_scaled,
    feature_names=features,
    show=False
)
plt.title("SHAP Summary — Random Forest")
plt.show()

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

<Figure size 640x480 with 0 Axes>

In [ ]:
# --- XGBoost SHAP ---
explainer_xgb = shap.TreeExplainer(models["XGBoost"])
shap_vals_xgb = explainer_xgb(X_scaled)

shap.summary_plot(
    shap_vals_xgb,
    X_scaled,
    feature_names=features,
    show=False
)
plt.title("SHAP Summary — XGBoost")
plt.show()

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("=== Cross-Validation Results ===")
for name, model in models.items():
    if name == "LogisticRegression":
        X_input = X_tr
        model_cv = model
    else:
        X_input = X_tr
        model_cv = model

    scores = cross_val_score(model_cv, X_input, y_tr, cv=cv, scoring="roc_auc")
    print(f"{name}: Mean ROC AUC = {scores.mean():.4f} (+/- {scores.std():.4f})")

In [ ]:
from xgboost import XGBClassifier

param_grid_log = {
    "logreg__penalty": ["l2", "l1"],
    "logreg__C": [0.01, 0.1, 1, 10],
}
logreg_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=9000, solver="saga"))
])
grid_log = GridSearchCV(logreg_pipeline, param_grid_log, cv=3, scoring="f1_macro", n_jobs=-1)
grid_log.fit(X_tr, y_tr)

best_logreg = grid_log.best_estimator_
print("Best Logistic Regression Params:", grid_log.best_params_)

param_grid_rf = {
    "n_estimators": [50, 100],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "class_weight": ["balanced"]
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3, scoring="f1_macro", n_jobs=-1)
grid_rf.fit(X_tr, y_tr)

best_rf = grid_rf.best_estimator_
print("Best Random Forest Params:", grid_rf.best_params_)

param_grid_xgb = {
    "n_estimators": [50, 100],
    "max_depth": [3, 6],
    "learning_rate": [0.05, 0.1],
}

grid_xgb = GridSearchCV(XGBClassifier(eval_metric="mlogloss"), param_grid_xgb, cv=3, scoring="f1_macro", n_jobs=-1)
grid_xgb.fit(X_tr, y_tr)

best_xgb = grid_xgb.best_estimator_
print("Best XGBoost Params:", grid_xgb.best_params_)